In [17]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs               
import cartopy.feature as cfeature         
import cartopy.util as cutil
import xarray as xr                        
import numpy as np
import pandas as pd
import xesmf as xe

In [18]:
ozone_dataset = xr.open_dataset("/glade/campaign/acom/acom-climate/UTLS/shawnh/archive/FCnudged_f09.mam.mar27.2000_2021.002/atm/proc/tseries/month_1/FCnudged_f09.mam.mar27.2000_2021.002.cam.h0.O3.200201-202412.nc")
pdeldry_dataset = xr.open_dataset("/glade/campaign/acom/acom-climate/UTLS/shawnh/archive/FCnudged_f09.mam.mar27.2000_2021.002/atm/proc/tseries/month_1/FCnudged_f09.mam.mar27.2000_2021.002.cam.h0.PDELDRY.200201-202412.nc")
ps_dataset = xr.open_dataset('/glade/campaign/acom/acom-climate/UTLS/shawnh/archive/FCnudged_f09.mam.mar27.2000_2021.002/atm/proc/tseries/month_1/FCnudged_f09.mam.mar27.2000_2021.002.cam.h0.PS.200201-202412.nc')

In [19]:
ozone = ozone_dataset["O3"]

In [20]:
p0 = ozone_dataset["P0"]
hyai = ozone_dataset["hyai"]
hybi = ozone_dataset["hybi"]
ps = ps_dataset['PS']
pdeldry = pdeldry_dataset['PDELDRY']
lev = ozone_dataset.coords['lev']
num_lev = lev.shape[0]

# convert to hPa from Pa
p0 = p0.copy() / 100
ps = ps.copy() / 100
pdeldry = pdeldry.copy() / 100 

# truncate to levels 16-31
truncated_pdeldry = pdeldry.isel({pdeldry.dims[1]: slice(10, 31)})
truncated_ozone = ozone.isel({ozone.dims[1]: slice(10, 31)})

Now group 'ozone' and 'pdeldry' DataArrays by months

In [21]:
start_date = '2005-02-01'
end_date = '2025-01-01'

# group the 240 month dates based on calendar months for both PDELDRY and O3 variables

truncated_pdeldry = truncated_pdeldry.sel(time=slice(start_date, end_date))
truncated_pdeldry = truncated_pdeldry.transpose('lev','time','lat','lon')

truncated_ozone = truncated_ozone.sel(time=slice(start_date, end_date))
#ozone_monthly_mean = truncated_ozone.groupby('time.month').mean('time')
truncated_ozone = truncated_ozone.transpose('lev','time','lat','lon')

In [22]:
# constants / conversion factor
NAv = 6.0221415e+23                       # molecules in mole
g = 9.81                                  # gravity
MWair = 28.94                             # g/mol
xp_const = (NAv * 10)/(MWair*g)           # scaling factor, pa to hPa and cm to m
DU_CONVERSION = 2.69 * 10**16

In [7]:
# Initialize pressure edge arrays
#mod_press_low = xr.zeros_like(ozone).transpose('lev','lat','lon','time')
mod_press_top = xr.zeros_like(ozone).transpose('lev','lat','lon','time')

Calculating pressure at hybrid levels

p(k) = a(k) * p0 + b(k) * ps

In [8]:
# Calculate pressure edge arrays
# CAM-chem layer indices start at the top and end at the bottom
for i in range(num_lev):
    mod_press_top[i,:,:,:] = hyai[i]*p0 + hybi[i]*ps

In [9]:
mod_press_top = mod_press_top.transpose('lev', 'time', 'lat', 'lon')

In [10]:
mod_press_top = mod_press_top.transpose('time', 'lat', 'lon', 'lev')
mod_press_top

<xarray.DataArray 'O3' (time: 276, lat: 192, lon: 288, lev: 32)> Size: 2GB
array([[[[   2.2552395,    5.031692 ,   10.157948 , ...,  651.1982   ,
           663.5244   ,  674.2979   ],
         [   2.2552395,    5.031692 ,   10.157948 , ...,  651.1982   ,
           663.5244   ,  674.2979   ],
         [   2.2552395,    5.031692 ,   10.157948 , ...,  651.1982   ,
           663.5244   ,  674.2979   ],
         ...,
         [   2.2552395,    5.031692 ,   10.157948 , ...,  651.1982   ,
           663.5244   ,  674.2979   ],
         [   2.2552395,    5.031692 ,   10.157948 , ...,  651.1982   ,
           663.5244   ,  674.2979   ],
         [   2.2552395,    5.031692 ,   10.157948 , ...,  651.1982   ,
           663.5244   ,  674.2979   ]],

        [[   2.2552395,    5.031692 ,   10.157948 , ...,  661.92914  ,
           674.53723  ,  685.55707  ],
         [   2.2552395,    5.031692 ,   10.157948 , ...,  661.6646   ,
           674.2657   ,  685.2795   ],
         [   2.2552395,    5.031692 ,   10.157948 , ...,  661.3697   ,
           673.9631   ,  684.9701   ],
...
         [   2.2552395,    5.031692 ,   10.157948 , ...,  959.5476   ,
           979.9721   ,  997.8238   ],
         [   2.2552395,    5.031692 ,   10.157948 , ...,  959.5462   ,
           979.97064  ,  997.82227  ],
         [   2.2552395,    5.031692 ,   10.157948 , ...,  959.54565  ,
           979.9701   ,  997.8217   ]],

        [[   2.2552395,    5.031692 ,   10.157948 , ...,  960.5574   ,
           981.00836  ,  998.8832   ],
         [   2.2552395,    5.031692 ,   10.157948 , ...,  960.5574   ,
           981.00836  ,  998.8832   ],
         [   2.2552395,    5.031692 ,   10.157948 , ...,  960.5574   ,
           981.00836  ,  998.8832   ],
         ...,
         [   2.2552395,    5.031692 ,   10.157948 , ...,  960.5574   ,
           981.00836  ,  998.8832   ],
         [   2.2552395,    5.031692 ,   10.157948 , ...,  960.5574   ,
           981.00836  ,  998.8832   ],
         [   2.2552395,    5.031692 ,   10.157948 , ...,  960.5574   ,
           981.00836  ,  998.8832   ]]]], dtype=float32)
Coordinates:
  * lat      (lat) float64 2kB -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon      (lon) float64 2kB 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * lev      (lev) float64 256B 3.643 7.595 14.36 24.61 ... 957.5 976.3 992.6
  * time     (time) datetime64[ns] 2kB 2002-02-01 2002-03-01 ... 2025-01-01
Attributes:
    mdims:         1
    units:         mol/mol
    mixing_ratio:  dry
    long_name:     O3 concentration
    cell_methods:  time: mean

In [14]:
filtered_300hpa_upper = mod_press_top.where(mod_press_top >= 300, drop=False)
filtered_300hpa_upper = filtered_300hpa_upper.where(mod_press_top < 322.24, drop=False)

In [15]:
mod_deltap = abs(300 - filtered_300hpa_upper)

In [16]:
ozone_filtered = ozone.where(mod_deltap.notnull())
ozone_filtered = ozone_filtered.transpose('lev','time','lat','lon')

In [17]:
mod_deltap_filtered = mod_deltap.where(ozone_filtered.notnull())

In [18]:
# check if data array is all NaN
mod_deltap_filtered.mean(dim={'lev', 'time','lat','lon'}, skipna=True)

<xarray.DataArray 'O3' ()> Size: 4B
array(12.481593, dtype=float32)

In [19]:
mod_deltap_filtered = mod_deltap_filtered.fillna(0)
ozone_filtered = ozone_filtered.fillna(0)

In [20]:
mod_deltap_filtered = mod_deltap_filtered.sel(time=slice(start_date, end_date))
ozone_filtered = ozone_filtered.sel(time=slice(start_date, end_date))

mod_deltap_filtered = mod_deltap_filtered.groupby('time.month').mean('time')
ozone_filtered = ozone_filtered.groupby('time.month').mean('time')

Calculating ozone column for [300hPa, 322.24hPa) and averaging it

In [21]:
ozone_300hpa_column = xr.dot(mod_deltap_filtered, xp_const*ozone_filtered, dims='lev')

In [22]:
ozone_du_300hpa_column = ozone_300hpa_column.copy() / DU_CONVERSION

ozone_du_300hpa_column = ozone_du_300hpa_column.where(ozone_du_300hpa_column != 0)

In [23]:
#ozone_du_300hpa_column = ozone_du_300hpa_column.sel(time=slice(start_date, end_date))
#ozone_du_300hpa_column = ozone_du_300hpa_column.groupby('time.month').mean('time')

Now calculating ozone column from 322.24hPa to ground level

In [24]:
ozone_part_column = xr.dot(truncated_pdeldry, xp_const*truncated_ozone, dims='lev')

In [25]:
ozone_du_column = ozone_part_column.copy() / DU_CONVERSION

In [26]:
ozone_du_column

<xarray.DataArray (time: 240, lat: 192, lon: 288)> Size: 53MB
array([[[ 77.88121 ,  77.88121 ,  77.88121 , ...,  77.88121 ,
          77.88121 ,  77.88121 ],
        [ 78.86642 ,  78.86391 ,  78.859795, ...,  78.87383 ,
          78.8717  ,  78.86872 ],
        [ 79.076035,  79.05303 ,  79.02679 , ...,  79.1448  ,
          79.12149 ,  79.098366],
        ...,
        [102.17463 , 102.16653 , 102.15504 , ..., 102.19886 ,
         102.18926 , 102.18179 ],
        [103.37187 , 103.36216 , 103.350105, ..., 103.40063 ,
         103.39009 , 103.38072 ],
        [105.32305 , 105.32305 , 105.32305 , ..., 105.32305 ,
         105.32305 , 105.32305 ]],

       [[ 75.89937 ,  75.89937 ,  75.89937 , ...,  75.89937 ,
          75.89937 ,  75.89937 ],
        [ 76.10659 ,  76.11236 ,  76.117226, ...,  76.0941  ,
          76.0973  ,  76.10107 ],
        [ 76.51267 ,  76.51832 ,  76.5242  , ...,  76.492256,
          76.49841 ,  76.5057  ],
...
        [105.68682 , 105.683136, 105.679985, ..., 105.702866,
         105.69792 , 105.692215],
        [106.16862 , 106.159325, 106.15086 , ..., 106.19918 ,
         106.18955 , 106.179565],
        [106.41641 , 106.41641 , 106.41641 , ..., 106.41641 ,
         106.41641 , 106.41641 ]],

       [[ 80.02395 ,  80.02395 ,  80.02395 , ...,  80.02395 ,
          80.02395 ,  80.02395 ],
        [ 80.3423  ,  80.32634 ,  80.31025 , ...,  80.39272 ,
          80.37635 ,  80.35889 ],
        [ 80.71654 ,  80.66684 ,  80.616806, ...,  80.86461 ,
          80.816574,  80.76698 ],
        ...,
        [ 95.70729 ,  95.71291 ,  95.71786 , ...,  95.687225,
          95.69464 ,  95.70123 ],
        [ 95.280785,  95.28573 ,  95.29007 , ...,  95.26353 ,
          95.269135,  95.27505 ],
        [ 94.76592 ,  94.76592 ,  94.76592 , ...,  94.76592 ,
          94.76592 ,  94.76592 ]]], dtype=float32)
Coordinates:
  * lat      (lat) float64 2kB -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon      (lon) float64 2kB 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * time     (time) datetime64[ns] 2kB 2005-02-01 2005-03-01 ... 2025-01-01

In [27]:
ozone_du_column.mean(dim={'time','lat','lon'}, skipna=True)

<xarray.DataArray ()> Size: 4B
array(66.36865, dtype=float32)

In [ ]:
e

Now calculate total tropospheric column ozone

In [ ]:
ozone_du_300hpa_column = ozone_du_300hpa_column.fillna(0)
total_tco = ozone_du_column + ozone_du_300hpa_column

In [ ]:
total_tco

Regridding and plotting

In [ ]:
omi_mls_ds = xr.open_dataarray("/glade/u/home/mvoncyga/SOARS_2025/OMIMLS_300hpa_monthly_mean_2005_2024_full.nc")
omi_mls_ds = omi_mls_ds.rename({'latitude': 'lat', 'longitude': 'lon'})

# shifting lon to be 0-360
omi_mls_ds['lon'] = omi_mls_ds['lon'] % 360
omi_mls_ds = omi_mls_ds.sortby('lon')

In [ ]:
regridder = xe.Regridder(ozone_du_column, omi_mls_ds, 'bilinear', periodic=True)  
cesm_regridded = regridder(total_tco)

In [ ]:
month_list = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

for i in range(12):
    plotting_module.plotting_ozone(cesm_regridded, i, "CESM $\mathregular{O_3}$ Concentration,", month_list[i], 2, 5, 40)

In [ ]:
difference = cesm_regridded - omi_mls_ds

print(difference.mean(dim={'month','lat','lon'}, skipna=True))

In [ ]:
nh_diff_mean = difference.sel(lat=slice(0,90)).mean(dim={'month','lat','lon'}, skipna=True)
sh_diff_mean = difference.sel(lat=slice(-90,0)).mean(dim={'month','lat','lon'}, skipna=True)

print(nh_diff_mean, sh_diff_mean)

In [ ]:
alphabet = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l'] # for cycling through figures to create a gif

i = 0
for i in range(12):
    
    plt.figure(figsize=(10, 6))
    ax = plt.axes(projection=ccrs.PlateCarree())
    difference[i].plot(ax=ax,transform=ccrs.PlateCarree(central_longitude=0), vmin = -12, vmax = 12, extend='both', cmap='bwr')
    ax.set_facecolor('gray')
    ax.add_feature(cfeature.COASTLINE, linestyle='--')
    ax.set_global()

    # adding gridlines
    ax.gridlines(draw_labels=True, color='black', alpha=0.5, linestyle='--')
    ax.set_title("CESM - OMI/MLS Tropo O3 Difference: " + month_list[i] + ", 2005-2024", fontsize=14)

    # adding textual info
    nh_formatted_mean = f"{difference[i].sel(lat=slice(0,90)).mean(dim={'lat','lon'}, skipna=True).data:.3f}"
    ax.text(0.10, -0.25, 'NH Mean: ' + str(nh_formatted_mean) + ' DU', va='bottom', ha='center',
            rotation='horizontal', rotation_mode='anchor',
            transform=ax.transAxes, fontsize=12)
    
    sh_formatted_mean = f"{difference[i].sel(lat=slice(-90,0)).mean(dim={'lat','lon'}, skipna=True).data:.3f}"
    ax.text(0.10, -0.35, 'SH Mean: ' + str(sh_formatted_mean) + ' DU', va='bottom', ha='center',
            rotation='horizontal', rotation_mode='anchor',
            transform=ax.transAxes, fontsize=12)
    

    #plt.savefig("figures/CESM_Diff-" + str(alphabet[i]))
    i+=1

    plt.show

In [ ]:
import imageio
import os

def create_gif(image_folder, output_gif, duration):
    filenames = sorted([f for f in os.listdir(image_folder) if os.path.isfile(os.path.join(image_folder, f))])
    images = []
    for filename in filenames:
        if filename.startswith("CESM_Diff"):
            image_path = os.path.join(image_folder, filename)
            images.append(imageio.imread(image_path))
    imageio.mimsave(output_gif, images, duration=duration)

image_dir = '/glade/u/home/mvoncyga/SOARS_2025/figures/'

#create_gif(image_dir, "Difference_Plot.gif", duration=1000)